<h1 style="text-align: center;">Bayesian programming Project</h1>
<p style="text-align: center;">SUPSI 3rd year Bachelor DS&AI</p>
<p style="text-align: center;">Tommaso Sommaruga, Miro Rava</p>

### `Introduction`
Football is a global phenomenon, with numerous championships showcasing distinct playing styles, team dynamics, and competitive structures. In this project, we focus on analyzing the differences and similarities between two of the most prominent football leagues: Serie A (Italy) and the Premier League (England).

The dataset used for this analysis is sourced from https://www.football-data.co.uk, a comprehensive repository of football match statistics. Each row in the dataset represents a matchday within a specific season for each championship with its related betting odds. While the dataset includes data from various global leagues, we have narrowed our scope to the Italian Serie A and the English Premier League to provide focused insights.

This project is structured around several statistical tasks aimed at uncovering meaningful patterns and testing hypotheses within the data:
- Exploratory Data Analysis: A detailed examination of the dataset to understand its structure and uncover initial trends.
- Hypothesis Testing: For each student, a hypothesis test will be conducted using Bayesian methods, with prior sensitivity analysis, posterior probability calculations (using ROPE), and a comparison with a frequentist approach.
- Regression Analysis: Each student will build a regression model, perform posterior predictive checks, and generate predictive distributions for test-set observations.
- Hierarchical vs. Unpooled Models: The group will construct both hierarchical and unpooled models, with a focus on comparing predictions for a specific group and analyzing the posterior distribution of a novel group.

---

## Data source
The following sources which have been utilised in the compilation of Football-Data's results and odds files.


Current results (full time, half time)
XScores - http://www.xscores .com

Match statistics
BBC, ESPN Soccer, Bundesliga.de, Gazzetta.it and Football.fr

Bookmakers betting odds
Betbrain.com
Oddsportal.com
Individual bookmakers

Betting odds for weekend games are collected Friday afternoons, and on Tuesday afternoons for midweek games.

### Dataset composition

In [ ]:
import pandas as pd
import os
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [3]:
#data_folder = "data"

# Initialize lists for Premier League and Serie A datasets
#premier_league_files = [f for f in os.listdir(data_folder) if f.startswith("E0") and f.endswith(".csv")]
#serie_a_files = [f for f in os.listdir(data_folder) if f.startswith("I1") and f.endswith(".csv")]

#def load_and_combine(files, league_name):
#    combined_df = pd.DataFrame()
#    for file in files:
#        file_path = os.path.join(data_folder, file)
#        df = pd.read_csv(file_path)
#
#        # Extract season from the filename (e.g., 'E0_2023.csv' -> '2023')
#        season = file.split("_")[1].split(".")[0]
#        df["Season"] = season
#        df["League"] = league_name
#        combined_df = pd.concat([combined_df, df], ignore_index=True)
#    return combined_df

#premier_league_data = load_and_combine(premier_league_files, "Premier League")
#serie_a_data = load_and_combine(serie_a_files, "Serie A")

#all_data = pd.concat([premier_league_data, serie_a_data], ignore_index=True)#

#display(all_data.head())

#all_data.to_csv("combined_league_data.csv", index=False)
#print("Combined dataset saved as 'combined_league_data.csv'.")


In [ ]:
data = pd.read_csv('combined_league_data.csv')

# Split data by league and season
serie_a_data = data[data['League'] == 'Serie A']
pl_data = data[data['League'] == 'Premier League']

display(serie_a_data.head())
display(pl_data.head())

In [ ]:
serie_a_data.columns

---

## Data cleaning

In [ ]:
serie_a_data.info()

In [ ]:
pl_data.info()

In [ ]:
data[data['Time'].isnull()]

In [ ]:
nan_data = data.columns[data.isna().sum() > 400]
nan_data

In [10]:
columns_to_drop = [
    'Time', 'Referee', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5',
    'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD', 'AvgCA',
    'Bb1X2', 'BbOU', 'BbAH', 'AHh', 'AHCh',
    'B365CH', 'BWCH', 'IWCH', 'WHCH', 'VCCH',
    'B365C>2.5', 'PC>2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5'
]
data = data.drop(columns=columns_to_drop)


In [ ]:
# Define bookmaker-specific columns for home, draw, and away odds
home_odds_columns = ['B365H', 'BFH', 'BSH', 'BWH', 'GBH', 'IWH', 'LBH', 'PSH', 'SOH', 'SBH', 'SJH', 'SYH', 'VCH', 'WHH']
draw_odds_columns = ['B365D', 'BFD', 'BSD', 'BWD', 'GBD', 'IWD', 'LBD', 'PSD', 'SOD', 'SBD', 'SJD', 'SYD', 'VCD', 'WHD']
away_odds_columns = ['B365A', 'BFA', 'BSA', 'BWA', 'GBA', 'IWA', 'LBA', 'PSA', 'SOA', 'SBA', 'SJA', 'SYA', 'VCA', 'WHA']

# Function to impute or derive maximum and average odds
def impute_or_derive_odds(data, max_col, avg_col, odds_columns):
    # Filter available odds columns
    available_odds = [col for col in odds_columns if col in data.columns]

    if not available_odds:
        raise ValueError(f"No bookmaker columns available to impute or derive {max_col} and {avg_col}.")

    # Impute missing values in available odds columns
    for col in available_odds:
        data[col] = data[col].fillna(data.groupby('Div')[col].transform('mean'))
        data[col] = data[col].ffill().bfill()

    # Derive Max and Avg odds
    data[max_col] = data[max_col].fillna(data[available_odds].max(axis=1))
    data[avg_col] = data[avg_col].fillna(data[available_odds].mean(axis=1))

# Impute or derive odds for home, draw, and away
impute_or_derive_odds(data, 'MaxH', 'AvgH', home_odds_columns)
impute_or_derive_odds(data, 'MaxD', 'AvgD', draw_odds_columns)
impute_or_derive_odds(data, 'MaxA', 'AvgA', away_odds_columns)

# Verify if there are any remaining NaN values in the critical columns
remaining_nans = data[['MaxH', 'AvgH', 'MaxD', 'AvgD', 'MaxA', 'AvgA']].isna().sum()
print("Remaining NaN values after imputation:")
print(remaining_nans)

In [ ]:
data.isna().sum()

In [ ]:
nan_cols = data.columns[(data.isna().sum() < 3) & (data.isna().sum() > 0)]
nan_cols

In [ ]:
data[data['BWH'].isnull()]

In [ ]:
data[data['B365H'].isnull()]

Cannot impute so, drop rows

In [16]:
odds_home = data['B365H'].values
data.loc[:, 'HomeWin'] = (data['FTR'] == 'H').astype(int)
home_win = data['HomeWin'].values

In [ ]:
nan_data = data.columns[data.isna().sum() > 400]
nan_data

In [18]:
data = data.drop(columns=nan_data)

In [ ]:
nan_cols = data.columns[data.isna().sum() > 0]
data[data[nan_cols].isnull().any(axis=1)]

This game had a result of 2-1 for Sassuolo.
The match was then awarded to Pescara by default, as the opposing team fielded a player who was not properly registered.

In [ ]:
match_condition = (data['HomeTeam'] == 'Sassuolo') & (data['AwayTeam'] == 'Pescara')

data.loc[match_condition, 'FTHG'] = 2
data.loc[match_condition, 'FTAG'] = 1
data.loc[match_condition, 'FTR'] = 'H'
data.loc[match_condition, 'HTHG'] = 1
data.loc[match_condition, 'HTAG'] = 0
data.loc[match_condition, 'HTR'] = 'H'

print(data.loc[match_condition, ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR']])


## Hypothesis test `1`
### see notebook click --> [HYPOTHESIS 1](hypothesis_test_1.ipynb) <--

## Hypothesis test `2`
### see notebook click --> [HYPOTHESIS 2](hypothesis_test_2.ipynb) <--

## Regression `1`: 
### see notebook click --> [REGRESSION 1](regression_1.ipynb) <--

## Regression `2`: 
### see notebook click --> [REGRESSION 2](regression_2.ipynb) <--

Bayesian Logistic Regression to predict the probability of a home team winning a game based on the odds. It assumes prior uncertainty about the model's coefficients (intercept and slope) and updates these beliefs using data. MCMC sampling is used to estimate the posterior distributions of the coefficients, giving us a range of possible values rather than fixed ones. This approach helps quantify uncertainty.

---

## MODELS POOLED, UNPOOLED, HIERARCHICAL

#### Plot the distribution of the data

In [22]:
data['goals_per_match'] = data['FTHG'] + data['FTAG']

In [23]:
goals_per_match_mean =data[['FTHG', 'FTAG']].mean(axis=1)
goals_per_match_std = data[['FTHG', 'FTAG']].std(axis=1)

In [ ]:
plt.figure(figsize=(10, 6))
goals_per_match = data["goals_per_match"]

sns.histplot(goals_per_match, kde=True, color='skyblue', bins=30)

# Adding titles and labels
plt.title('Distribution of Goals per Match', fontsize=16)
plt.xlabel('Goals per Match', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

# Calculating and displaying interesting statistics
mean_goals = np.mean(goals_per_match)
std_goals = np.std(goals_per_match)
median_goals = np.median(goals_per_match)
min_goals = np.min(goals_per_match)
max_goals = np.max(goals_per_match)

# Displaying the stats on the plot
stats_text = f'Mean: {mean_goals:.2f}\nMedian: {median_goals:.2f}\nStd Dev: {std_goals:.2f}\nMin: {min_goals}\nMax: {max_goals}'
plt.text(0.95, 0.95, stats_text, horizontalalignment='right', verticalalignment='top',
         transform=plt.gca().transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.7))

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
pl_goals_per_match = pl_data['FTHG'] + pl_data['FTAG']
serie_a_goals_per_match = serie_a_data['FTHG'] + serie_a_data['FTAG']

# Calculate statistics for Premier League
pl_mean_goals = pl_goals_per_match.mean()
pl_median_goals = pl_goals_per_match.median()
pl_std_goals = pl_goals_per_match.std()
pl_min_goals = pl_goals_per_match.min()
pl_max_goals = pl_goals_per_match.max()

# Calculate statistics for Serie A
serie_a_mean_goals = serie_a_goals_per_match.mean()
serie_a_median_goals = serie_a_goals_per_match.median()
serie_a_std_goals = serie_a_goals_per_match.std()
serie_a_min_goals = serie_a_goals_per_match.min()
serie_a_max_goals = serie_a_goals_per_match.max()

# Plotting the distribution of goals per match for Premier League and Serie A
plt.figure(figsize=(12, 6))
sns.histplot(pl_goals_per_match, kde=True, color='blue', label='Premier League', bins=30)
sns.histplot(serie_a_goals_per_match, kde=True, color='red', label='Serie A', bins=30)

# Adding titles and labels
plt.title('Distribution of Goals per Match: Premier League vs. Serie A', fontsize=16)
plt.xlabel('Goals per Match', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.legend()

# Displaying the stats on the plot
stats_text = f'Premier League:\nMean: {pl_mean_goals:.2f}, Median: {pl_median_goals:.2f}, Std Dev: {pl_std_goals:.2f}, Min: {pl_min_goals}, Max: {pl_max_goals}\n\nSerie A:\nMean: {serie_a_mean_goals:.2f}, Median: {serie_a_median_goals:.2f}, Std Dev: {serie_a_std_goals:.2f}, Min: {serie_a_min_goals}, Max: {serie_a_max_goals}'
plt.text(0.95, 0.95, stats_text, horizontalalignment='right', verticalalignment='top',
            transform=plt.gca().transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.7))

# Show the plot
plt.tight_layout()
plt.show()



#### Choice of the prior based on online checks

- link to source Serie A: [soccerzz.com](https://www.soccerzz.com/competition/serie-a/10/stats)
- link to source Premier League: [soccerzz.com](https://www.soccerzz.com/competition/premier-league/4/stats)


In [26]:
all_time_mean_pl = 2.97
all_time_mean_sa = 2.62

all_time_std_pl = 1.2
all_time_std_sa = 1.2

# Model pooled

In [27]:
# derive the pooled prior mean and standard deviation
pooled_mean = (all_time_mean_pl + all_time_mean_sa) / 2
pooled_std = np.sqrt(all_time_std_pl**2 + all_time_std_sa**2)

In [ ]:
# Build the PyMC model
with pm.Model() as pooled_model:
    # Prior for the mean goals per match (using a Normal distribution)
    mu = pm.Normal('mu', mu=pooled_mean, sigma=pooled_std)

    # Prior for the standard deviation (positive only) using HalfNormal
    sigma = pm.HalfNormal('sigma', sigma=pooled_std)

    # Likelihood: Goals per match follow a Normal distribution
    y = pm.Normal('y', mu=mu, sigma=sigma, observed=data["goals_per_match"])
    
    # Sample the posterior
    pooled_trace = pm.sample(2000, return_inferencedata=True)

In [ ]:
with pooled_model:
    prior_pred = pm.sample_prior_predictive()
pm.plot_posterior(prior_pred)

In [ ]:
with pooled_model:
    posterior_pred = pm.sample_posterior_predictive(pooled_trace)

In [ ]:
pm.summary(pooled_trace)

# Unpooled Model

In [ ]:
grouped_data = data.groupby(['Season', 'League'])['goals_per_match']

with pm.Model() as unpooled_model:
    lambda_dict = {}

    # Loop through each group (season and league)
    for (season, league), group_data in grouped_data:
        # Prior for the Poisson rate (lambda) for each group
        lambda_dict[(season, league)] = pm.HalfNormal(f'lambda_{season}_{league}', sigma=goals_per_match_std)

        # Likelihood for the goals in each match in the group (Poisson distribution)
        pm.Poisson(f'y_{season}_{league}', mu=lambda_dict[(season, league)], observed=group_data)

    # Sampling from the model
    unpooled_trace = pm.sample(10_000, return_inferencedata=True)

In [ ]:
pm.plot_trace(unpooled_trace)
plt.show()

pm.summary(unpooled_trace)

# Hierarchical model

In this case, you want to account for differences between teams (like in the "factory machine" example).

The idea is to model the goal counts for each team, while allowing for some "sharing" of information between teams based on their historical performance (across seasons).

Model description:
- The mean goal counts for each team ($\mu_j$) are drawn from a global mean ($\mu_\mu$).
- The global mean of goal counts, $\mu_\mu$, is assumed to be normally distributed with a prior belief, N(2, 1) for the mean number of goals scored per match.
- The standard deviation ($\sigma$) reflects the variance of goals scored for each match, shared across teams.
- The goal counts for each match ($y_ij$) are modeled as coming from a normal distribution with a team-specific mean ($\mu_j$) and a global standard deviation ($\sigma$).


In [ ]:
goals_per_match_mean = data["goals_per_match"].mean()
goals_per_match_std = data["goals_per_match"].std()

# Calculate team-level mean and standard deviation
team_stats = data.groupby("HomeTeam")["FTHG"].agg(["mean", "std"]).reset_index()
team_stats.rename(columns={"mean": "team_mean", "std": "team_std"}, inplace=True)

goals_per_match_mean, goals_per_match_std

In [ ]:
team_stats

In [ ]:
team_stats.plot(kind='scatter', x='team_mean', y='team_std', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
global_mean = team_stats['team_mean'].mean()
global_std = team_stats['team_mean'].std()

global_mean, global_std

In [ ]:
teams = data['HomeTeam'].unique()
n_teams = len(teams)

# Map team names to indices for the model
team_idx = data['HomeTeam'].map({team: idx for idx, team in enumerate(teams)}).values

with pm.Model() as hierarchical_model:
    # Global mean and standard deviation
    mu_mu = pm.Normal("mu_mu", mu=global_mean, sigma=global_std)  # Use global_mean and global_std
    sigma_global = pm.HalfNormal("sigma_global", sigma=global_std)  # Prior based on global_std

    # Team-level means
    mu_team = pm.Normal("mu_team", mu=mu_mu, sigma=sigma_global, shape=n_teams)

    # Match-level standard deviation
    sigma = pm.HalfNormal("sigma", sigma=global_mean)  # Can remain the same

    # Observed goals per match
    y_obs = pm.Normal("y_obs", mu=mu_team[team_idx], sigma=sigma, observed=data['goals_per_match'])

    # Sampling the model
    trace_hierarchical = pm.sample(2_000, return_inferencedata=True)

# Posterior summary
pm.summary(trace_hierarchical)


In [ ]:
az.summary(trace_hierarchical)
az.plot_posterior(trace_hierarchical)

In [ ]:
with hierarchical_model:
    posterior_predictive = pm.sample_posterior_predictive(trace_hierarchical)

In [ ]:
posterior_goals = posterior_predictive['observed_data']
posterior_goals

In [ ]:
predicted_goals = posterior_predictive["posterior_predictive"]
predicted_goals = predicted_goals['y_obs']

In [99]:
from bokeh.io import output_notebook, show
output_notebook()

In [ ]:
plt.figure(10, 6)
az.plot_ppc(posterior_predictive, var_names=["y_obs"])
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
az.plot_ppc(posterior_predictive, var_names=["y_obs"], backend="bokeh")
plt.show()